# Siamese Neural Network - MNIST dataset

## Import Library

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds

## Data process

### Load and split dataset

In [11]:
ds, ds_info = tfds.load('mnist', split=['train','test'], shuffle_files=True, as_supervised=True, with_info=True)
ds_train = ds[0]
ds_test = ds[1]


In [16]:
ds_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\mithr\\tensorflow_datasets\\mnist\\3.0.1',
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

### Build training pipeline

In [12]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
  normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

TypeError: in user code:


    TypeError: tf__normalize_img() missing 1 required positional argument: 'label'


### Build evaluation pipeline

In [6]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

# Model Training

### Build the model

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])

### Compile the model

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


### Model fitting

In [10]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 5s 3ms/step - loss: 0.3589 - sparse_categorical_accuracy: 0.9016 - val_loss: 0.1895 - val_sparse_categorical_accuracy: 0.9461
Epoch 2/6
469/469 [==============================] - 1s 2ms/step - loss: 0.1649 - sparse_categorical_accuracy: 0.9536 - val_loss: 0.1343 - val_sparse_categorical_accuracy: 0.9606
Epoch 3/6
469/469 [==============================] - 1s 2ms/step - loss: 0.1190 - sparse_categorical_accuracy: 0.9654 - val_loss: 0.1172 - val_sparse_categorical_accuracy: 0.9646
Epoch 4/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0914 - sparse_categorical_accuracy: 0.9730 - val_loss: 0.0996 - val_sparse_categorical_accuracy: 0.9707
Epoch 5/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0746 - sparse_categorical_accuracy: 0.9777 - val_loss: 0.0899 - val_sparse_categorical_accuracy: 0.9709
Epoch 6/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0618 - sparse_categorical_accu

### Plot acuracy and loss 